<a href="https://colab.research.google.com/github/AishJo/Repository1/blob/master/Text_Analytics_Category_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
!pip install -U imbalanced-learn

     |████████████████████████████████| 163kB 4.9MB/s 
  Found existing installation: imbalanced-learn 0.4.3
    Uninstalling imbalanced-learn-0.4.3:
      Successfully uninstalled imbalanced-learn-0.4.3


In [4]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input
from keras.layers import Dense, Input, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Embedding
import keras.backend as K 
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phraser, Phrases

Using TensorFlow backend.


In [5]:
%%time
# reading the data
df = pd.read_csv('/content/drive/Shared drives/DSO 560 NLP Project/data.csv')
# creating a subset of the relevant attribute name and then dropping the column
df = df[df['attribute_name'] == 'category'].drop(columns = ['attribute_name'])
# combining similar category attribute values and removing spaces
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'sweatshirthoodie' if x == 'sweatshirt hoodie' else x)
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'blazerscoatsjackets' if x == 'blazer , coat jacket' else x)

CPU times: user 408 ms, sys: 77.9 ms, total: 486 ms
Wall time: 1.13 s


In [6]:
df.head()

,product_id,brand,product_full_name,description,brand_category,attribute_value,details
3,01DVCVGDJQC4J8JJ35PKZ4J169,loewe,hammock small textured leather shoulder bag,black textured leather calf hook fasten open w...,bag tote bag tote bag,accessory,item 's measurement depth 14 cm height 23 cm m...
32,01DPGSTG4M1RXB26QMMN0MPPB8,j crew,mockneck sweater supersoft yarn,cozy flattering mockneck sweater feel good loo...,sweater,sweater,NaN
36,01E2KKRH99SZ1PTR2E2ABVYJ5S,alo,vapor high waist legging,signature lift fabric moisture wicking technol...,unknown,NaN,"xxs 00 0 , xs 0 2 , s 4 6 , m 8 10 , l 12 14 ,..."
50,01E2M1MYJ1FYVHG2AMXHDMNQSP,alo,prism high waist capris,high waistband help flat figure moisture wicki...,unknown,NaN,"xxs 00 , xs 0 2 , s 4 6 , m 8 10 , l 12 14 hig..."
52,01E2KZ8BZF6M1XCHBZGEX3AF78,alo,cover tank,hard earn abs supersoft faux wrap tank 's crop...,unknown,NaN,"xxs 00 0 , xs 0 2 , s 4 6 , m 8 10 , l 12 14 ,..."


In [0]:
# creating a feature combining brand, productname, description and brand category
df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']).apply(str)

In [0]:
# the predictor 'text' is assigned to 'X' and the 'attribute_value' t 'y'
X = df['text'].values
# one-hot-encoding the y variable
y = pd.get_dummies(df['attribute_value']).values

In [0]:
# creating a function to create uni-grams and tokenizing data
def encode_1gram(X, mode = 'binary'):
    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [0]:
# creating a function to create bi-grams and tokenizing data
def encode_2gram(X, mode='binary'):
    phrases = Phrases(X, min_count=30)
    bigrams = Phraser(phrases)
    X = list(bigrams[X])

    tokenizer = Tokenizer(num_words=500)
    tokenizer.fit_on_texts(X)
    length = max([len(s.split()) for s in df['text']])
    X = tokenizer.texts_to_matrix(X)
    X = pad_sequences(X, maxlen=length, padding='post')
    vocab_size = len(tokenizer.word_index) + 1
    return X, length, vocab_size

In [11]:
Tokenizer()

In [0]:
# creating tf-idf vectors from uni-gram encoding
X1, length1, vocab_size1 = encode_1gram(X, mode = 'tfidf')

In [0]:
# creating tf-idf vectors from bi-gram encoding
X2, length2, vocab_size2 = encode_2gram(X, mode = 'tfidf')

In [14]:
X1.shape

(4194, 155)

In [0]:
# concatenating uni-gram and bi-gram
X = np.concatenate([X1, X2], axis = 1)

In [0]:
# splitting data into training and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [0]:
# balancing the sample sets as the proportion of labels is unbalanced
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [0]:
num_classes = y.shape[1]

In [0]:
# defining LSTM model with sigmoid activation functions for prediction
def define_model():
    inputs = Input(shape=(length1+length2,))
    embedding = Embedding(input_dim=vocab_size1, output_dim=100)(inputs)

    x = LSTM(16, return_sequences=True, dropout=0.2, recurrent_dropout=0.15)(embedding)

    x = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(x)
    x = Conv1D(filters=num_classes, kernel_size=310, padding='valid')(x)
    x = Reshape((num_classes,))(x)
    out = Activation('sigmoid')(x)

    model = Model(inputs = [inputs], outputs = out)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [20]:
model = define_model()
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 310)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 310, 100)          615000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 310, 16)           7488      
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 310, 7)            119       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 1, 7)              15197     
_________________________________________________________________
reshape_1 (Reshape)          (None, 7)                 0         
_________________________________________________________________
activation_1 (Activation)    (None, 7)                 0   

In [21]:
# fitting data onto model
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 11143 samples, validate on 2786 samples
Epoch 1/10
11143/11143 [==============================] - 20s 2ms/step - loss: 0.4777 - accuracy: 0.8234 - val_loss: 0.6970 - val_accuracy: 0.8571
Epoch 2/10
11143/11143 [==============================] - 13s 1ms/step - loss: 0.3953 - accuracy: 0.8571 - val_loss: 0.7205 - val_accuracy: 0.8571
Epoch 3/10
11143/11143 [==============================] - 14s 1ms/step - loss: 0.3912 - accuracy: 0.8571 - val_loss: 0.6997 - val_accuracy: 0.8571
Epoch 4/10
11143/11143 [==============================] - 13s 1ms/step - loss: 0.3907 - accuracy: 0.8571 - val_loss: 0.7024 - val_accuracy: 0.8571
Epoch 5/10
11143/11143 [==============================] - 13s 1ms/step - loss: 0.3905 - accuracy: 0.8571 - val_loss: 0.6854 - val_accuracy: 0.8571
Epoch 6/10
11143/11143 [==============================] - 13s 1ms/step - loss: 0.3902 - accuracy: 0.8571 - val_loss: 0.6957 - val_accuracy: 0.8571
Epoch 7/10
11143/11143 [==============================] - 13s 1ms/ste

In [22]:
# evaluating model
model.evaluate(X_test, y_test)

420/420 [==============================] - 1s 2ms/step


[0.30307326316833494, 0.914625883102417]

In [23]:
# #predictions for each class from predicting model
pd.Series(model.predict(X).argmax(axis = -1)).value_counts()

3    1435
0    1397
4     697
1     366
2     299
dtype: int64

In [24]:
# probability of predicting each class for various records
model.predict(X)

array([[0.25260967, 0.13102639, 0.21596007, ..., 0.12782818, 0.09172553,
        0.01040277],
       [0.25350934, 0.2271456 , 0.12138864, ..., 0.14013225, 0.14065358,
        0.01887008],
       [0.1256843 , 0.28149217, 0.12203814, ..., 0.22928755, 0.07950082,
        0.0088543 ],
       ...,
       [0.21815757, 0.20690438, 0.15427567, ..., 0.1493864 , 0.09867868,
        0.01287725],
       [0.48520407, 0.09497063, 0.13583376, ..., 0.2575071 , 0.14270839,
        0.03054205],
       [0.14194198, 0.1515477 , 0.18994842, ..., 0.15998942, 0.08698082,
        0.00868193]], dtype=float32)

In [25]:
# split of classes
pd.Series(y.argmax(axis = -1)).value_counts()

0    2184
4     672
2     427
1     353
5     295
6     146
3     117
dtype: int64